In [ ]:
path_home="C:/Usees/KANKI/Documents/github/habmap"
path_file_face="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/faces_dataset.csv"
path_file_v5="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/vertice_dataset.csv"
path_file_v1="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/vertice_dataset_raw.csv"
table_f=CSV.read(path_file_face,header=false, DataFrame)
table_v5=CSV.read(path_file_v5,header=false, DataFrame)
table_v1=CSV.read(path_file_v1,header=false, DataFrame)

table_f=convert(Matrix,table_f)
table_v5=convert(Matrix,table_v5)
table_v1=convert(Matrix,table_v1)
row=size(table_f)[1]

function calcs03B(table_f, table_v5, table_v1, iter)
    row, col=size(table_f)
    row_v1, col_v1=size(table_v1)
    
    g=zeros(Float32,row_v1,3)  
    terrain_variables=zeros(Float32, row, 12)

    bottom_all=findmin(table_v5[:,3])[1]    
    for c=1:row
        for d=1:3
            g[c,d]=(table_v5[table_f[c,1]+1,d]+table_v5[table_f[c,2]+1,d]+table_v5[table_f[c,3]+1,d])/3
        end
    end
    
    for c=1:row
        terrain_variables[c,1]=g[c,3]
        terrain_variables[c,2]=g[c,3]-bottom_all
    end
    
    for c=1:iter
        distance=zeros(Float32, row_v1, 2)
        for i2=1:row_v1
            distance[i2,1]=(table_v1[i2,1]-g[c,1])^2+(table_v1[i2,2]-g[c,2])^2
            distance[i2,2]=distance[i2,1]+(table_v1[i2,3]-g[c,3])^2
        end
        
        kernels=[0.01, 0.04, 0.16, 0.64]
        
        #Calculating rugosity and bpi
        
        for (countk, kernel) in enumerate(kernels)
            index_p=findall(distance[:,1] .< kernel)
            ver_in_ker_p=zeros(Float32, size(index_p)[1], 1)
            for (i, index) in enumerate(index_p)
                ver_in_ker_p[i,1]=table_v1[index[1],3]
            end 
            
            index_v=findall(distance[:,2] .< kernel)
            ver_in_ker=zeros(Float32, size(index_v)[1], 3)
            for (i, index) in enumerate(index_v)
                ver_in_ker[i,1:3]=table_v1[index[1],1:3]
            end
            
            bottom=findmin(ver_in_ker_p[:,1])[1]
            meanz=Statistics.mean(ver_in_ker_p[:,1])
            terrain_variables[c,countk+2]=g[c,3]-bottom
            terrain_variables[c,countk+6]=g[c,3]-meanz
            M=fit(PCA,ver_in_ker)
            distances=zeros(Float32, size(index_v)[1], 1)
            
        end
    end
    return terrain_variables
end

@time calcs03B(table_f, table_v5, table_v1, 200)